In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

Vamos definir as funções utilizadas nessa avaliação.

In [2]:
# Função para solucionar um sistema em que a matriz de coeficientes é do tipo triangular inferior
def sol_triang_inf(A, B):
  # A é uma matriz triangular inferior e B um vetor com os termos independentes do sistema
  n = len(B)
  X = []
  for line in range(n):
    b = B[line]
    for column in range(line):
      b -= A[line][column]*X[column]
    x = b/A[line][line]
    # Obtemos os valores de X por substiruições recursivas
    X.append(x)
  return np.array(X)
  # Retornamos o vetor X

In [3]:
# Função para transformar uma matriz triangular superior em uma triangular inferior
def inverte_matriz(A):
  # A é uma matriz triangular superior
  new_A = []
  for array in A[::-1]: # Inverte a ordem das linhas de A
    new_A.append(array[::-1]) # Inverte a ordem dos elementos de cada linha de A
  return np.array(new_A)

In [4]:
# Função para solucionar um sistema em que a matriz de coeficientes é do tipo triangular superior
def sol_triang_sup(A,B):
  # A é uma matriz triangular superior e B um vetor com os termos independentes do sistema
  A = inverte_matriz(A) # Transforma A em uma matriz triangular inferior
  B = B[::-1] # Inverte o vetor B
  return sol_triang_inf(A,B)[::-1] # Soluciona o sistema obtido e inverte os valores de X obtidos

In [5]:
# Função para realizar o pivoteamento de uma coluna
def pivot(A, B, n):
  # A é a matriz de coeficientes do sistema, B um vetor com os termos independentes do sistema e n a coluna à qual será aplicado o pivoteamento
  index_pivo = abs(A[n:,n]).argsort()[-1] + n
  # Encontra a linha do elemento de maior módulo da coluna n buscando a partir da linha n
  aux = np.copy(A[n])
  A[n] = A[index_pivo]
  A[index_pivo] = aux
  # Troca a linha n e a linha com o novo pivô
  aux = B[n]
  B[n] = B[index_pivo]
  B[index_pivo] = aux
  # Aplica a mesma troca ao vetor B
  return A, B # Retorna a matriz A e o vetor B modificados

In [6]:
# Método de Gauss
def gauss(A, b, pivotar=False, returnL = False):
  # A é a matriz de coeficientes do sistema, b um vetor com os termos independentes do sistema, pivotar é uma variável que indica se deve ou não se utilizada
  # a estratégia de pivoteamento simples e returnL indica se devem ser retornadas as matrizes L e U para o método LU
  n = len(A)
  A = A.astype(np.float64) 
  b = b.astype(np.float64)
  # Definindo o tipo dos dados como ponto flutuante, para ser possível operar sobre um vetor inteiro de uma só vez
  B = np.copy(b) # Criando uma cópia do vetor B para não alterar o original
  if returnL:
    L = np.identity(n)
  for j in range(n-1): # Itera sobre as colunas de A, exceto a última
    if pivotar:
      A, B = pivot(A, B, j)
    for i in range(j+1,n): # Itera sobre as linhas de A a partir de j+1
      fator = A[i][j]/A[j][j]
      B[i] = B[i] - B[j]*fator # Onde B[i] e B[j] são reais
      A[i] = A[i] - A[j]*fator # Onde A[i] e A[j] são vetores
      if returnL:
        L[i][j] = fator
  if returnL:
    return L, A # Retorna as matrizes L e U se returnL = True
  return sol_triang_sup(A, B) # Retorna a solução do sistema se returnL = False

In [7]:
# Método de Gauss-Jordan
def gauss_jordan(A, B, pivotar=False):
  # A é a matriz de coeficientes do sistema, b um vetor com os termos independentes do sistema e pivotar é uma variável que indica se deve ou não se utilizada
  # a estratégia de pivoteamento simples 
  n = len(A)
  A = A.astype(np.float64) 
  B = B.astype(np.float64)
  # Definindo o tipo dos dados como ponto flutuante, para ser possível operar sobre um vetor inteiro de uma só vez
  inversa = np.identity(n) # Inicializando a matriz inversa de A como a identidade de ordem n
  for j in range(n): # Itera sobre as colunas de A
    if pivotar:
      A, B = pivot(A, B, j)
    else: 
      inversa[j] = inversa[j]/A[j][j] # Só faz sentido calcular a inversa se não houver pivoteamento
    B[j] = B[j]/A[j][j]
    A[j] = A[j]/A[j][j]
    # Definindo o termo da diagonal pertencente a linha j igual a 1
    for i in range(n): # Itera sobre as linhas de A
      if i != j:
        B[i] = B[i] - B[j]*A[i][j]
        inversa[i] = inversa[i] - inversa[j]*A[i][j]
        A[i] = A[i] - A[j]*A[i][j]
        # Definindo os termos que não são da dioganal principal pertencentes à coluna j iguais a 0
  if pivotar:
    return B
  return B, inversa # Retorna o vetor B (que é igual a X) e a matriz inversa obtida, se não for realizado pivoteamento

In [8]:
# Método de Gauss-Seidel
def gauss_seidel(A, B, erromax, maxiter=50, X=np.zeros(3), iter=0):
  # A é a matriz de coeficientes do sistema, B um vetor com os termos independentes do sistema, erromax é o erro máximo tolerado, maxiter é o maior numero
  # de iterações permitido, X é o vetor solução atual e iter é o número de iterações realizadas
  A = A.astype(np.float64)
  B = B.astype(np.float64)
  # Definindo o tipo dos dados como ponto flutuante, para ser possível operar sobre um vetor inteiro de uma só vez
  n = len(A)
  if X.all() == 0:
    X = np.copy(B)
  X_antigo = np.copy(X)
  for i in range(n):
    X[i] = (-np.dot(A[i], X.T) + A[i][i]*X[i] + B[i])/A[i][i]
    # Calculando os novos valores de X, utilizando o vetor X já atualizado
  erro = max(abs(X_antigo - X))/max(abs(X)) # Calculando o erro obtido
  iter+=1
  if erro <= erromax or iter == maxiter:
    return X, iter # Retorna o vetor X que obteve erro tolerável ou foi obtido após o número máximo de iterações e o número de iterações realizadas
  return gauss_seidel(A, B, erromax, maxiter, X, iter) # Chama a própria função recursivamente caso os critérios de parada não tenham sido atingidos

In [9]:
# Função para calcular o resíduo
def residuo(A, x, B):
  # A é a matriz de coeficientes do sistema, x é o vetor solução obtido por algum dos métodos e B é o vetor de termos independentes dos sistema
  return np.dot(A, x) - B

**QUESTÃO 1**

Primeiramente, vamos definir a matriz de coeficientes e o vetor de termos independentes do sisteam

In [10]:
A = np.array([[80,0,30,10],[0,80,10,10],[16,20,60,72],[4,0,0,8]])
B = np.array([40,27,31,2])

Em seguida, vamos utilizar o método de Gauss-Seidel utilizando os parâmetros pedidos

In [11]:
x, iter = gauss_seidel(A, B, 0.005, X=np.array([0.5,0.2,0.2,0]))
x

array([0.3998614 , 0.30000386, 0.25075653, 0.0500693 ])

In [12]:
iter

14

Como podemos observar, o método levou 14 iterações para encontrar os valores acima. Considerando o erro tolerado, podemos assumir que a solução para o sistema é x1 = 0.4, x2 = 0.3, x3 = 0.25 e x4 = 0.05. Vamos, por fim, verificar o resíduo de tal solução

In [13]:
residuo(A, x, B)

array([0.01230089, 0.00856708, 0.0482413 , 0.        ])

Como podemos observar, o resíduo está dentro da tolenrância desejada

**QUESTÃO 2**

Sejam x1, x2, x3 e x4 a quantidade de computadores do tipo 1, 2, 3 e 4, respectivamente, obtemos o seguinte sistema de equações lineares:

*3x1 + 4x2 + 7x3 + 20x4 = 504*

*20x1 + 25x2 + 40x3 + 50x4 = 1970*

*10x1 + 15x2 + 20x3 +22x4 = 970*

*10x1 + 8x2 + 10x3 + 15x4 = 601*

Definindo a matriz de coeficientes e o vetor de termos independentes do sistema:

In [14]:
A = np.array([[3, 4, 7, 20],[20,25,40,50], [10,15,20,22], [10,8,10,15]])
B = np.array([504,1970, 970, 601])

Agora, vamos utilizar o método da Eliminação de Gauss para solucionar o sistema

In [15]:
x = gauss(A, B)
x

array([10., 12., 18., 15.])

Ou seja, serão produzidos 10 computadores do tipo 1, 12 do tipo 2, 18 do tipo 3 e 15 do tipo 4. Vamos, por fim, verificar o resíduo da nossa solução

In [16]:
residuo(A, x, B)

array([0.00000000e+00, 0.00000000e+00, 2.27373675e-13, 2.27373675e-13])

Obtivemos um resíduo aproximadamente nulo, confirmando a corretude da solução obtida

**QUESTÂO 3**

Substituindo os valores informados pelo enunciado, encontramos o seguinte sistema:

*636 - T = 70a*

*518 + T - R = 60a*

*307 + R = 40a*

Somando as 3 equações, obtemos:

*1461 = 170a*, logo a = 8.594 m\s², aproximadamente. Assim, obtemos o seguinte sistema linear:

*-T = -34.42*

*T - R = -2.34*

*R = 36,76*

Ou seja, T vale aproximadamente 34,42N e R vale 36,76N. Podemos também encontrar esses valores utilizando um sistema com apenas as duas primeiras equações e utilizando o método de Gauss para solucioná-lo

In [17]:
A = np.array([[-1,0], [1, -1]])
B = np.array([-34.42, -2.34])
x = gauss(A, B)
x

array([34.42, 36.76])

E vamos, por fim, calcular o resíduo obtido:

In [18]:
residuo(A, x, B)

array([ 0.00000000e+00, -3.55271368e-15])

Tal resíduo deve-se à aproximação feita no valor da aceleração

**QUESTÃO 4**

Substituindo os valores informados pelo enunciado, obtemos o seguinte sistema de equações lineares:

*-200x1 + 50x2 = 0*

*50x1 -125x2 + 75x3 = 0*

*75x2 - 300x3 + 225x4 = 0*

*-225x3 + 225x4 = 2000*

Vamos, agora, definir a matriz dos coeficientes e o vetor dos termos independentes do sistema:

In [19]:
A = np.array([[-200,50,0,0],[50,-125,75,0],[0,75,-300,225],[0,0,-225,225]])
B = np.array([0,0,0,2000])

E utilizar o método da Eliminação de Gauss para solucionar o sistema:

In [20]:
x = gauss(A, B)
x

array([13.33333333, 53.33333333, 80.        , 88.88888889])

Ou seja, x1 = 13.33, x2 = 53.33, x3 = 80 e x4 = 88.88, todos os valores em metros. Finalmente, vamos calcular o resíduo para a solução obtida:

In [21]:
residuo(A, x, B)

array([0., 0., 0., 0.])